In [1]:
!pip install -q transformers accelerate spacy torch numpy scikit-learn
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 104.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [8]:
import torch
import spacy
import logging
import re
import numpy as np
import collections
import string
from typing import List, Tuple, Optional, Dict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("OptimizedQAGS")

class OptimizedQAGS:
    def __init__(self, device_id: int = 0, batch_size: int = 8, use_fp16: bool = True):
        self.device = device_id
        self.batch_size = batch_size
        self.device_str = f"cuda:{device_id}" if device_id >= 0 else "cpu"
        self.torch_dtype = torch.float16 if use_fp16 and device_id >= 0 else torch.float32

        # Load spacy
        self.nlp = spacy.load("en_core_web_sm", disable=["lemmatizer", "textcat"])

        # Question generation model
        self.qg_model_name = "valhalla/t5-base-qg-hl"
        self.qg_tokenizer = AutoTokenizer.from_pretrained(self.qg_model_name)
        self.qg_model = AutoModelForSeq2SeqLM.from_pretrained(
            self.qg_model_name,
            torch_dtype=self.torch_dtype
        ).to(self.device_str)
        self.qg_model.eval()

        # Question answering pipeline - use better model for harder questions
        self.qa_pipeline = pipeline(
            "question-answering",
            model="deepset/roberta-base-squad2",
            tokenizer="deepset/roberta-base-squad2",
            device=self.device,
            handle_impossible_answer=True,
            batch_size=self.batch_size
        )

    def extract_candidates(self, text: str, max_candidates: int = 25) -> List[Dict[str, any]]:
        """Extract answer candidates with context for better QG."""
        text = (text or "").strip()
        if not text:
            return []

        doc = self.nlp(text)
        candidates = []
        seen = set()

        # High-priority entities for factual claims
        priority_labels = {
            "PERSON": 10, "ORG": 9, "GPE": 8, "LOC": 7,
            "DATE": 9, "CARDINAL": 8, "MONEY": 8,
            "PERCENT": 8, "EVENT": 8, "WORK_OF_ART": 7,
            "ORDINAL": 7, "TIME": 8, "QUANTITY": 7
        }

        # Extract named entities
        for ent in doc.ents:
            if ent.label_ in priority_labels:
                cand_text = ent.text.strip()
                cand_lower = cand_text.lower()
                if cand_text and cand_lower not in seen and len(cand_text.split()) <= 8:
                    seen.add(cand_lower)
                    candidates.append({
                        'text': cand_text,
                        'start': ent.start_char,
                        'end': ent.end_char,
                        'priority': priority_labels[ent.label_],
                        'type': ent.label_
                    })

        # Add noun chunks for additional coverage
        if len(candidates) < max_candidates:
            stop_words = {"it", "he", "she", "they", "this", "that", "these",
                         "those", "we", "you", "i", "me", "my", "your", "his",
                         "her", "our", "their", "its"}

            for chunk in doc.noun_chunks:
                clean = chunk.text.strip()
                clean_lower = clean.lower()
                words = clean.split()

                # Better filtering
                if (clean and
                    clean_lower not in seen and
                    clean_lower not in stop_words and
                    2 <= len(words) <= 6 and
                    not all(w.lower() in stop_words for w in words)):

                    # Skip if it's mostly punctuation or numbers
                    if sum(c.isalnum() for c in clean) > len(clean) * 0.5:
                        seen.add(clean_lower)
                        candidates.append({
                            'text': clean,
                            'start': chunk.start_char,
                            'end': chunk.end_char,
                            'priority': 4,
                            'type': 'NOUN_CHUNK'
                        })

                        if len(candidates) >= max_candidates:
                            break

        # Sort by priority
        candidates.sort(key=lambda x: x['priority'], reverse=True)
        return candidates[:max_candidates]

    def highlight_answer_in_context(self, context: str, answer: str,
                                     start: int, end: int) -> str:
        """Create highlighted context for QG."""
        before = context[:start]
        highlighted = f"<hl> {answer} <hl>"
        after = context[end:]
        return before + highlighted + after

    def generate_questions_batch(self, summary: str, candidates: List[Dict],
                                 min_questions: int = 10, max_questions: int = 20) -> List[Tuple[str, str]]:
        """Generate high-quality questions."""
        summary = (summary or "").strip()
        if not summary or not candidates:
            return []

        # Prepare inputs
        input_texts = []
        valid_candidates = []

        for cand in candidates[:max_questions * 2]:
            highlighted = self.highlight_answer_in_context(
                summary,
                cand['text'],
                cand['start'],
                cand['end']
            )
            input_texts.append(highlighted)
            valid_candidates.append(cand)

        if not input_texts:
            return []

        # Generate questions
        inputs = self.qg_tokenizer(
            input_texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        ).to(self.device_str)

        with torch.no_grad():
            outputs = self.qg_model.generate(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                max_length=64,
                num_beams=5,  # Increased for better quality
                early_stopping=True,
                no_repeat_ngram_size=3,
                length_penalty=1.2,  # Encourage longer questions
                num_return_sequences=1
            )

        questions = self.qg_tokenizer.batch_decode(outputs, skip_special_tokens=True)

        # Filter and validate
        qa_pairs = []
        seen_questions = set()

        question_starters = {
            "what", "who", "where", "when", "why", "how", "which",
            "whose", "whom", "did", "does", "do", "is", "are", "was",
            "were", "has", "have", "had", "can", "could", "will", "would",
            "should", "may", "might"
        }

        for q, cand_dict in zip(questions, valid_candidates):
            q_clean = q.strip()
            if not q_clean:
                continue

            # Add question mark if missing
            if not q_clean.endswith("?"):
                q_clean = q_clean + "?"

            q_lower = q_clean.lower()
            words = q_clean.split()

            # Must be reasonable length
            if len(words) < 4 or len(words) > 20:
                continue

            # Must start with question word
            first_word = words[0].lower().strip("?.,!")
            if first_word not in question_starters:
                continue

            # Skip degenerate outputs
            if q_lower.strip("?") in {"true", "false", "yes", "no"}:
                continue

            # Check answer visibility in question
            cand_text = cand_dict['text']
            cand_lower = cand_text.lower()
            q_no_punct = re.sub(r'[^\w\s]', ' ', q_lower).strip()

            # For named entities and important facts, allow even if answer appears
            # For generic chunks, be more strict
            if cand_dict['type'] in ['NOUN_CHUNK']:
                if cand_lower in q_no_punct and len(words) < 8:
                    continue

            # Deduplicate similar questions
            q_normalized = ' '.join(sorted(q_lower.split()))
            if q_normalized not in seen_questions:
                seen_questions.add(q_normalized)
                qa_pairs.append((q_clean, cand_text))

                if len(qa_pairs) >= max_questions:
                    break

        return qa_pairs

    def get_answers_batch(self, context: str, questions: List[str]) -> List[Tuple[str, float]]:
        """Get answers with confidence scores."""
        context = (context or "").strip()
        if not context or not questions:
            return [("", 0.0)] * len(questions)

        try:
            preds = self.qa_pipeline(
                question=questions,
                context=[context] * len(questions),
                doc_stride=128,
                max_seq_len=512,
                max_answer_len=50,
                top_k=1
            )
        except Exception as e:
            logger.warning(f"QA pipeline error: {e}")
            return [("", 0.0)] * len(questions)

        if not isinstance(preds, list):
            preds = [preds]

        answers = []
        for p in preds:
            score = float(p.get("score", 0.0))
            ans = (p.get("answer") or "").strip()

            # Return answer with confidence
            if score < 0.01 or not ans or ans.lower() in {"no answer", "unknown", "none"}:
                answers.append(("", 0.0))
            else:
                answers.append((ans, score))

        return answers

    def compute_f1(self, a_gold: str, a_pred: str) -> Optional[float]:
        """Compute F1 with exact match bonus."""
        def normalize(s):
            s = re.sub(r'\b(a|an|the)\b', ' ', s.lower())
            s = ''.join(ch if ch not in string.punctuation else ' ' for ch in s)
            return ' '.join(s.split())

        a_gold = (a_gold or "").strip()
        a_pred = (a_pred or "").strip()

        # Both empty = can't evaluate
        if not a_gold and not a_pred:
            return None

        # One empty = complete mismatch
        if not a_gold or not a_pred:
            return 0.0

        gold_norm = normalize(a_gold)
        pred_norm = normalize(a_pred)

        if not gold_norm and not pred_norm:
            return None

        if not gold_norm or not pred_norm:
            return 0.0

        # Exact match after normalization
        if gold_norm == pred_norm:
            return 1.0

        gold_toks = gold_norm.split()
        pred_toks = pred_norm.split()

        # Token-level F1
        common = collections.Counter(gold_toks) & collections.Counter(pred_toks)
        num_same = sum(common.values())

        if num_same == 0:
            return 0.0

        precision = num_same / len(pred_toks)
        recall = num_same / len(gold_toks)
        f1 = (2 * precision * recall) / (precision + recall)

        return f1

    def calculate_score(self, source: str, summary: str, verbose: bool = False,
                       confidence_weight: bool = True) -> float:
        """
        Calculate QAGS score with optional confidence weighting.

        Args:
            source: Source document
            summary: Summary to evaluate
            verbose: Print detailed output
            confidence_weight: Weight scores by QA confidence (helps detect hallucinations)
        """
        source = (source or "").strip()
        summary = (summary or "").strip()

        if not source or not summary:
            logger.warning("Empty source or summary")
            return 0.0

        # Extract candidates from summary
        candidates = self.extract_candidates(summary, max_candidates=25)

        if verbose:
            print(f"Extracted {len(candidates)} candidates")
            print(f"Top candidates: {[c['text'] for c in candidates[:8]]}\n")

        if not candidates:
            logger.warning("No candidates extracted from summary")
            return 0.0

        # Generate questions
        qa_pairs = self.generate_questions_batch(summary, candidates,
                                                  min_questions=10, max_questions=20)

        if verbose:
            print(f"Generated {len(qa_pairs)} questions\n")

        if not qa_pairs:
            logger.warning("No valid questions generated")
            return 0.0

        # Get answers with confidence
        questions = [q for q, _ in qa_pairs]
        expected_answers = [a for _, a in qa_pairs]

        ans_src = self.get_answers_batch(source, questions)
        ans_sum = self.get_answers_batch(summary, questions)

        # Calculate F1 scores
        scores = []
        confidences = []

        for i, (q, exp_ans, (a_sum, conf_sum), (a_src, conf_src)) in enumerate(
            zip(questions, expected_answers, ans_sum, ans_src)):

            f1 = self.compute_f1(a_src, a_sum)

            if verbose:
                print(f"{i+1}. Q: {q}")
                print(f"   Expected: {exp_ans}")
                print(f"   Summary: '{a_sum}' (conf: {conf_sum:.2f})")
                print(f"   Source:  '{a_src}' (conf: {conf_src:.2f})")

                if f1 is None:
                    print(f"   F1: None (both empty)")
                else:
                    print(f"   F1: {f1:.2f}")

                # Highlight potential hallucinations
                if f1 is not None and f1 < 0.3 and conf_sum > 0.5:
                    print(f"   ⚠️  POTENTIAL HALLUCINATION (low F1, high summary confidence)")
                print()

            if f1 is not None:
                scores.append(f1)
                # Use minimum confidence (if either is low, penalize)
                min_conf = min(conf_src, conf_sum) if confidence_weight else 1.0
                confidences.append(min_conf)

        if not scores:
            logger.warning("No valid F1 scores computed")
            return 0.0

        # Calculate weighted average if using confidence
        if confidence_weight and sum(confidences) > 0:
            # Weight by confidence - low confidence answers get less weight
            weighted_scores = [s * (0.5 + 0.5 * c) for s, c in zip(scores, confidences)]
            final_score = float(np.mean(weighted_scores))
        else:
            final_score = float(np.mean(scores))

        if verbose:
            print(f"{'='*70}")
            print(f"Final QAGS Score: {final_score:.3f}")
            print(f"Valid comparisons: {len(scores)}/{len(qa_pairs)}")
            avg_f1 = np.mean(scores)
            avg_conf = np.mean(confidences) if confidences else 0
            print(f"Average F1: {avg_f1:.3f} | Average Confidence: {avg_conf:.3f}")
            print(f"{'='*70}")

        return final_score

In [9]:
qags = OptimizedQAGS(batch_size=16, use_fp16=True)

source_text = """
Lucas Matthysse won a majority decision against Ruslan Provodnikov in a 12-round super lightweight bout on Saturday night. Matthysse landed the majority of the punches in the first round and opening a cut near Provodnikov's left eye early in the second. Provodnikov (24-3) put Matthysse on the ropes late in the third round and landed two hard right hook-left hook combos in the fourth before Matthysse (37-3) regained control in the fifth. He continued to use his three-inch reach advantage to keep Provodnikov at bay, giving him room to dodge the Russian's powerful left hook. Lucas Matthysse celebrates after his win against Ruslan Provodnikov at the Turning Stone Resort Casino . Provodnikov (right) lands an uppercut to the head of Matthysse despite having a cut opened up early on . Provodnikov (24-3) put Matthysse on the ropes late in the third round and landed two hard right hook-left hook combos in the fourth before Matthysse (37-3) regained control in the fifth. He continued to use his three-inch reach advantage to keep Provodnikov at bay, giving him room to dodge the Russian's powerful left hook. Provodnikov landed several punches in the later rounds but couldn't knock down his Argentine opponent down, although he did stagger Matthysse in the 11th. 'He did, he hurt me,' Matthysse said through a translator. 'But I was able to withstand the onslaught. He's a very tough fighter. He's very strong. He just keeps coming forward.' Don Ackerman scored the fight as a draw 114-114, but Glenn Feldman and John McKaie both scored it 115-113 in favor of Matthysse. 'To me, he was better today,' Provodnikov said through a translator. 'He was the better man in the ring and, you know, it was a close fight but he won and I hope everybody enjoyed it.' The night's undercard was quickly decided when Patrick Teixeira (25-0, 21 KOs) won with a second-round knockout of Patrick Allotey (30-2) in their middleweight bout. Provodnikov admitted that Matthysse was better than him on the day in the majority decision win . Matthysse said he just wanted to rest but talked up a fight with Floyd Mayweather or Manny Pacquiao next . The sell-out crowd then had to wait more than an hour for the main event, watching the TV feed of Terence Crawford beating Thomas Dulorme for the vacant WBO junior welterweight title in Austin, Texas. WBO president Paco Valcarcel tweeted he would like to see Crawford defend his title against Matthysse, but the Argentinian had his sights set higher. 'For right now I just want to rest,' Matthysse said. 'I got my daughter, my family waiting for me back home. I want to rest. I want to go back there and see them so much and let's see what happens with (Manny) Pacquiao and (Floyd) Mayweather.' Matthysse and Provodnikov prepare to hug at the end of their fight on Saturday night in Verona, New York ."""

summary_1 = """
Lucas Matthysse won a majority decision against Ruslan Provodnikov in a 12-round super lightweight bout on Saturday night . Matthysse landed the majority of the punches in the first round and opening a cut near Provodnikov's left eye early in the second . Provodnikov (24-3) put Matthysse on the ropes late in the third round and landed two hard right hook-left hook combo
"""

summary_2 = """
Lucas Matthysse beat Ruslan Provodnikov 1-0 at the Turning Stone Resort Casino. Matthysse opened a cut near Provodnikov's left eye early in the second round. Patrick Teixeira (25-0, 21 KOs) won with a second-round knockout of Patrick Allotey (30-2)
"""

def interpret(s):
    if s>0.9: return "High Faithfulness"
    if s>0.6: return "Mixed/Partial"
    return "Low Faithfulness"

print("Summary 1:")
s1 = qags.calculate_score(source_text, summary_1, verbose=True)
print("Score:", s1, interpret(s1))

print("\nSummary 2:")
s2 = qags.calculate_score(source_text, summary_2, verbose=True)
print("Score:", s2, interpret(s2))

Device set to use cuda:0


Summary 1:
Extracted 22 candidates
Top candidates: ['Ruslan Provodnikov', 'Lucas Matthysse', '12', 'Saturday night', 'Provodnikov', '24-3', 'Matthysse', 'two']

Generated 16 questions

1. Q: Who did Lucas Matthysse fight on Saturday night?
   Expected: Ruslan Provodnikov
   Summary: 'Ruslan Provodnikov' (conf: 0.99)
   Source:  'Ruslan Provodnikov' (conf: 0.95)
   F1: 1.00

2. Q: Who won a majority decision against Ruslan Provodnikov?
   Expected: Lucas Matthysse
   Summary: 'Lucas Matthysse' (conf: 0.99)
   Source:  'Lucas Matthysse' (conf: 0.90)
   F1: 1.00

3. Q: How many rounds did the bout last?
   Expected: 12
   Summary: '12' (conf: 0.91)
   Source:  '12' (conf: 0.91)
   F1: 1.00

4. Q: Who did Matthysse open a cut near in the second round?
   Expected: Provodnikov
   Summary: 'Provodnikov's left eye' (conf: 0.32)
   Source:  'Provodnikov's left eye' (conf: 0.59)
   F1: 1.00

5. Q: What was Provodnikov's record against Matthysse?
   Expected: 24-3
   Summary: '24-3' (conf: 0.62)

In [14]:
src2 = "Manchester City Under 18s boss Jason Wilcox, whose side take on Chelsea in the first leg of the FA Youth Cup Final on Monday, is aiming to emulate United's famous Class of '92. Wilcox, himself let go by his current employers as a teenager, could field a starting XI of players all born in Greater Manchester for the clash against the favoured opponents from west London. And, while it may be unlikely anyone will emulate a golden generation across town that yielded the likes of the Neville brothers, Paul Scholes, Nicky Butt, Ryan Giggs and David Beckham, the ex-Blackburn winger believes that is the target. Manchester City Under 18s manager Jason Wilcox wants to deliver local players to the first team . Wilcox talks with Manchester City elite development squad coach Patrick Vieira . 'It could happen again but they were freak years,' Wilcox said. 'You look at United as a one-off, Barcelona the same. 'Our job is to recruit local lads. If we can get a group of Manchester-born lads into the first team then that's the ideal scenario. To do that is going to be really difficult but that's the challenge.' The first leg will take place under lights at the impressive, newly-opened 7,200-capacity stadium within the grounds of the £200m Etihad Campus, a sprawling complex aimed at kickstarting a conveyor belt of homegrown talent and defying those who claim the club have no interest nurturing growth from within. Wilcox believes those facilities, coupled with elite coaching, stand City in good stead when it comes to recruiting talent. Wilcox hopes to emulate Manchester United's 'Class of 92' but says it will be very difficult . 'We don't want local lads playing for Tottenham, Arsenal or Chelsea,' he said. 'It's up to us to make sure that, via the scouting, we have at least the opportunity of showing them our programme. I'm convinced once they see us it's very difficult to walk away from.' Will the final, between two clubs often criticised for spending millions of foreign players, be a chance to stick two fingers up to Football Association chairman Greg Dyke, who a year ago described the lack of English talent at Manchester City as 'pretty depressing'? Wilcox does not want to get into that. 'There’s a lot of media attention from Greg Dyke with regards to increasing the home-grown quota,' he said. Wilcox believes City's new Etihad campus will stand club in good stead when it comes to recruiting youngsters . A look inside one of the gyms at Manchester City's training complex . 'We can’t get embroiled in that because, for us, it’s a day-to-day question of how do we improve players? 'Proposals have already been put forward with regards to home-grown quotas. If he (Dyke) wants to increase it, then that is up to him. But whether you increase or lower the numbers, it doesn’t change the fact that you have to be good enough to get through.' Wilcox knows all about being good enough to get through at a club where money is no object. The Bolton youngster, who joined Blackburn at 14, was the only youth product to regularly appear in their Jack Walker-fuelled Premier League triumph in 1995. 'When I was a young player at Blackburn Jack Walker came in and brought Kenny (Dalglish),' he said. Manchester City's Isaac Buckley Ricketts (left) celebrates scoring during the semi-final against Leicester . 'You go two ways. Do I go elsewhere and take my chances or have I got stomach for fight and see what happens? Luckily for me I was the only homegrown player which was a great privilege. Now I'm passing my experience to younger ones.' Wilcox's arrival at Ewood Park came after rejection at none other than City. He remembers it well. 'I’d joined City at 13 and got released at 14,' he recalled. 'It’s quite fitting really because when I’m speaking to younger players I can always recall that chat if you like. I think at that time it was a case of I just got told that they were not going to retain me on a year’s contract. 'It was devastating at the time. Everything was going really well. But to be told that it is tough for a young kid. But I just recall my own experience whenever I have to tell the lads and have the younger age groups. You have to go through some period in your life like that where not everything is going to go fantastically well.' Wilcox was one of the only graduate players to feature regularly for Blackburn during title-winning season . City manager Manuel Pellegrini will be at the Etihad Campus, as will England Under 21s chief Gareth Southgate. Things have not gone according to plan for the Chilean's first team this season, but does Wilcox believe their struggles have increased the pressure on his youngsters to deliver the club's only silverware this season? 'Not really,' he said. 'At the start of the year, irrespective of what was happening at first-team level, the FA Youth Cup was a chance for our players to be put in a situation where they are being put under a bit of pressure and I think it’s great that they are being put under a bit of pressure.' Aided by Wilcox's wise head, they appear to be thriving under it."

In [15]:
summary2_normal = "Manchester City Under 18s face Chelsea in the FA Youth Cup Final on Monday . Jason Wilcox wants to bring local players to the first team . Wilcox says it will be difficult but he hopes to emulate Class of '92 ."

In [16]:
summary2_simple = "Jason Wilcox could field a starting XI of players born in Greater Manchester . City face Chelsea in the first leg of the FA Youth Cup final on monday . Wilcox believes City's new Etihad campus will stand them in good stead when it comes to recruiting youngsters ."

In [17]:
print("Summary 2 Normal:")
s1 = qags.calculate_score(src2, summary2_normal, verbose=True)
print("Score:", s1, interpret(s1))

print("\nSummary 2 Simple:")
s2 = qags.calculate_score(src2, summary2_simple, verbose=True)
print("Score:", s2, interpret(s2))

Summary 2 Normal:
Extracted 11 candidates
Top candidates: ['Chelsea', 'Jason Wilcox', '18s', 'Monday', 'Wilcox', "'92", 'Manchester City', 'the FA Youth Cup Final']

Generated 7 questions

1. Q: Who will the Manchester City Under 18s face in the FA Youth Cup Final on Monday?
   Expected: Chelsea
   Summary: 'Chelsea' (conf: 0.99)
   Source:  'Chelsea' (conf: 0.94)
   F1: 1.00

2. Q: Who wants to bring local players to the first team?
   Expected: Jason Wilcox
   Summary: 'Jason Wilcox' (conf: 0.98)
   Source:  'Jason Wilcox' (conf: 0.30)
   F1: 1.00

3. Q: What age group will Manchester City Under play Chelsea in the FA Youth Cup Final?
   Expected: 18s
   Summary: '18' (conf: 0.44)
   Source:  '18s' (conf: 0.46)
   F1: 0.00

4. Q: Who says he hopes to emulate Class of '92?
   Expected: Wilcox
   Summary: 'Jason Wilcox' (conf: 0.88)
   Source:  'Jason Wilcox' (conf: 0.72)
   F1: 1.00

5. Q: What class did Jason Wilcox want to emulate?
   Expected: '92
   Summary: 'Class of '92' (conf: 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Generated 11 questions

1. Q: Who believes City's new Etihad campus will stand them in good stead when recruiting youngsters?
   Expected: Jason Wilcox
   Summary: 'Jason Wilcox' (conf: 0.62)
   Source:  'Wilcox' (conf: 0.85)
   F1: 0.67

2. Q: Who will City face in the FA Youth Cup final on monday?
   Expected: Chelsea
   Summary: 'Chelsea' (conf: 0.96)
   Source:  'Chelsea' (conf: 0.94)
   F1: 1.00

3. Q: What could Jason Wilcox field a starting ?
   Expected: XI
   Summary: 'XI of players born in Greater Manchester' (conf: 0.34)
   Source:  'Wilcox, himself let go by his current employers as a teenager, could field a starting XI of players all born in Greater Manchester' (conf: 0.13)
   F1: 0.50

4. Q: What is the name of the competition in which Manchester City play Chelsea?
   Expected: the FA Youth Cup
   Summary: 'FA Youth Cup final' (conf: 0.55)
   Source:  'FA Youth Cup Final' (conf: 0.69)
   F1: 1.00

5. Q: Whose new Etihad campus does Wilcox believe will stand them in good s